<a href="https://colab.research.google.com/github/lamtung1997/Solving_PDEs_Deep_learning/blob/main/1_Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# SETUP
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time
import math
import itertools
pi = math.pi

np.random.seed(1)
tf.random.set_seed(1)

In [2]:
# MODEL
nn = 32
inputs = keras.Input(shape=(2,), name='points')
hidden_1 = layers.Dense(nn, activation='tanh', name='hidden_1')(inputs)
outputs = layers.Dense(1, activation='linear', name="u")(hidden_1)
model = keras.Model(inputs=inputs, outputs=outputs, name='stokes')
model.summary()

Model: "stokes"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
points (InputLayer)          [(None, 2)]               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 32)                96        
_________________________________________________________________
u (Dense)                    (None, 1)                 33        
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


In [3]:
# DATA
import itertools

n = 40                      # number of points in a border
c = np.linspace(0, 1, n)    # coordinates from 0 to 1 with uniform distribution
P_in, P_border = [],[]
for i, j in itertools.product(range(n), range(n)):
    p = [c[i], c[j]]
    if(i == 0 or i == n-1 or j == 0 or j == n-1):
        P_border.append(p)
    else:
        P_in.append(p)

# upsampling P_border that its size is equal to P_in
for i in range(len(P_in)-len(P_border)):
    j = i%(len(P_border))
    P_border.append(P_border[j])

# set batch_size and shuffle_size
batch_size = 1
bs = int(len(P_border))

# convert numpy array, P_in and P_border to Tensorflow Dataset
P_in = tf.convert_to_tensor(P_in, dtype=tf.float32)
P_border = tf.convert_to_tensor(P_border, dtype=tf.float32)

P_in = tf.data.Dataset.from_tensor_slices((P_in))
P_in = P_in.shuffle(buffer_size=bs).batch(batch_size)

P_border = tf.data.Dataset.from_tensor_slices((P_border))
P_border = P_border.shuffle(buffer_size=bs).batch(batch_size)

In [4]:
print("Number of data: ", int(P_in.cardinality()*batch_size*2))

Number of data:  2888


In [5]:
# CHOOSE OPTIMIZER
optimizer = tf.keras.optimizers.Adam()

In [6]:
# TRAIN_STEP FUNCTION
@tf.function
def train_step(P_in, P_border):
    with tf.GradientTape(persistent=True) as tape:
        loss_in, loss_border = [0], [0]
        # interior loss
        for i in range(P_in.shape[0]):
            v = tf.reshape(P_in[i], shape=(1,2))
            tape.watch(v)
            model_vals = model(v)

            # calculating u_xx and u_yy
            u_x_y = tape.gradient(model_vals, v)
            u_xx = tape.gradient(u_x_y[0][0], v)[0][0]
            u_yy = tape.gradient(u_x_y[0][1], v)[0][1]

            # calculating f(x,y)
            f = -2*pi*pi* tf.math.sin(pi*v[0][0]) * tf.math.sin(pi*v[0][1])

            # calculating loss_in
            loss_in += tf.math.square(u_xx + u_yy - f)

        # boundary loss
        for j in range(P_border.shape[0]):
            v = tf.reshape(P_border[j], shape=(1,2))
            loss_border += tf.math.square(model(v))
        
        # loss function
        batch_loss = loss_in + loss_border
    
    # update weights
    grads = tape.gradient(batch_loss, model.trainable_weights)          # gradient of loss function with respect to w
    optimizer.apply_gradients(zip(grads, model.trainable_weights))      # update w

    return batch_loss

In [7]:
# TRAIN FUNCTION
losses = []
def train(learning_rate):
    optimizer.learning_rate.assign(learning_rate)
    loss = 0
    for (P_in_batch,P_border_batch) in itertools.zip_longest(P_in,P_border):
        batch_loss = train_step(P_in_batch, P_border_batch)
        loss += batch_loss
    losses.append(loss.numpy())

In [8]:
# TRAIN
i = 0
max_ite = 1000
lr = 1e-3
start = time.time()
print("Learning rate:", lr)
while(True):
    start_ite = time.time()
    train(lr)
    if(i%1 == 0):
        print("[%3s] loss = %13.7f \t %5.3fs \t less than %5.3f minutes to finish" % (i, losses[i], time.time() - start_ite, (time.time() - start_ite)*(max_ite - i)/60))
    if(i > 200):
        if(losses[i] > np.min(losses[i-200: i-100])):
            lr = lr/10
            if(lr < 1e-5):
                break
            print("\nNew learning rate:", lr)
    if(i > max_ite):
        break
    i = i + 1


print("Total time: %.2f minutes" % ((time.time() - start)/60))

Learning rate: 0.001
[  0] loss = 58270.4414062 	 1.842s 	 less than 30.698 minutes to finish
[  1] loss = 18492.3984375 	 1.016s 	 less than 16.920 minutes to finish
[  2] loss =  9830.2675781 	 1.113s 	 less than 18.517 minutes to finish
[  3] loss =  5102.1650391 	 1.149s 	 less than 19.090 minutes to finish
[  4] loss =  3324.1711426 	 1.103s 	 less than 18.317 minutes to finish
[  5] loss =  2887.7377930 	 1.104s 	 less than 18.305 minutes to finish
[  6] loss =  2647.5292969 	 1.123s 	 less than 18.611 minutes to finish
[  7] loss =  2454.5834961 	 1.121s 	 less than 18.551 minutes to finish
[  8] loss =  2248.3703613 	 1.077s 	 less than 17.798 minutes to finish
[  9] loss =  2007.6070557 	 1.093s 	 less than 18.057 minutes to finish
[ 10] loss =  1754.9333496 	 1.095s 	 less than 18.069 minutes to finish
[ 11] loss =  1473.1275635 	 1.100s 	 less than 18.128 minutes to finish
[ 12] loss =  1268.6977539 	 1.119s 	 less than 18.418 minutes to finish
[ 13] loss =  1068.4681396 	 1

In [9]:
def u(network,x,y):
    z = tf.convert_to_tensor([[x, y]])
    u = float(network(z))
    return u

In [10]:
model.save("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson")

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson/assets


In [11]:
model1 = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson")

In [12]:
n = 10                      # number of points in a border
c = np.linspace(0, 1, n)    # coordinates from 0 to 1 with uniform distribution
P = []
for i, j in itertools.product(range(n), range(n)):
    p = [c[i], c[j]]
    P.append(p)

def exact_sol(x,y):
    return math.sin(pi*x)*math.sin(pi*y)

err = 0
for i in range(len(P)):
    err += (u(model1, P[i][0], P[i][1]) - exact_sol(P[i][0], P[i][1]))**2

L2 = math.sqrt(err/len(P))
print("L2 = ", L2)

L2 =  0.002438958730443497
